# Wyszukiwanie w tekstach

Kluczowym problemem nlp jest wyszukiwanie podobnych dokumentów.

# Zad 
Prosze wczytać słownik i korpus.

In [10]:
from gensim import corpora, models, similarities, matutils
import numpy as np
import os 

In [11]:
# documents = ["Romeo and Juliet",
#          "Juliet: O happy dagger",
#          "Romeo died by dagger",
#          "'Live free or die', that’s the New-Hampshire’s motto",
#          "Did you know, New-Hampshire is in New-England"]

In [12]:
if (os.path.exists("tmp/deerwester.dict")):
    dictionary = corpora.Dictionary.load('tmp/deerwester.dict')
    corpus = corpora.MmCorpus('tmp/deerwester.mm')
    print("Used files generated from first tutorial")
else:
    print("Please run first tutorial to generate data set")



Used files generated from first tutorial


In [13]:
for d in corpus:
    print(d)

[(0, 1.0), (1, 1.0)]
[(2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0)]
[(1, 1.0), (2, 1.0), (6, 1.0), (7, 1.0)]
[(8, 1.0), (9, 1.0), (10, 1.0), (11, 1.0), (12, 1.0), (13, 1.0), (14, 1.0)]
[(15, 1.0), (16, 1.0), (17, 1.0), (18, 1.0), (19, 1.0), (20, 1.0)]


In [14]:
scipy_csc_matrix = matutils.corpus2csc(corpus)
print(scipy_csc_matrix.todense())

[[1. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0.]
 [0. 1. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


In [15]:
print(dictionary.token2id)

{'juliet': 0, 'romeo': 1, 'dagger': 2, 'happy': 3, 'juliet:': 4, 'o': 5, 'by': 6, 'died': 7, "'live": 8, "die',": 9, 'free': 10, 'motto': 11, 'new-hampshire’s': 12, 'or': 13, 'that’s': 14, 'did': 15, 'is': 16, 'know,': 17, 'new-england': 18, 'new-hampshire': 19, 'you': 20}


# Zad 
Proszę znaleść reprezentację dokumentu:
```python
doc = "died dagger"
```

In [16]:
doc = "died dagger"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_bow

[(2, 1), (7, 1)]

# Wyszukiwanie najbliższych dokumentów

* W gensim najpierw inicjalizujemy query (tworzymy macierz podobieństw) a potem możemy querować podobne dokumenty. 

* Do querowania używane jest podobieństwo cosinusowe.


In [17]:
index = similarities.MatrixSimilarity(corpus)
print(list(index))

[array([0.99999994, 0.        , 0.35355338, 0.        , 0.        ],
      dtype=float32), array([0.  , 1.  , 0.25, 0.  , 0.  ], dtype=float32), array([0.35355338, 0.25      , 1.        , 0.        , 0.        ],
      dtype=float32), array([0.        , 0.        , 0.        , 0.99999994, 0.        ],
      dtype=float32), array([0., 0., 0., 0., 1.], dtype=float32)]


In [18]:
sims = index[vec_bow] # perform a similarity query against the corpus
print(list(enumerate(sims))) # print (document_number, document_similarity) 2-tuples

[(0, 0.0), (1, 0.35355338), (2, 0.70710677), (3, 0.0), (4, 0.0)]


# Zad 
Proszę posortować korpus od najbardziej pasujących zdań do najmniej pasujących do naszego quary

In [19]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims) # print sorted (document number, similarity score) 2-tuples

[(2, 0.70710677), (1, 0.35355338), (0, 0.0), (3, 0.0), (4, 0.0)]


# Zad 
Proszę przanalizować wynik. Jakie dokumenty okazały się najbardziej podobne?

# Zad 
Proszę użyć tfidf do analogicznego zadania. 

* Należy stworzyć odpowiednie funkcje które pozwolą nam używać różnych transformacji do querowania.

Czy wynik się różni? Co się zmieniło?

In [23]:
tfidf = models.TfidfModel(corpus, id2word=dictionary)
corpus_tfidf = tfidf[corpus]

vec_tfidf = tfidf[vec_bow]

index = similarities.MatrixSimilarity(corpus_tfidf)

sims = index[vec_tfidf]
print(list(enumerate(sims)))

[(0, 0.0), (1, 0.15449485), (2, 0.7071067), (3, 0.0), (4, 0.0)]


In [24]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims)

[(2, 0.7071067), (1, 0.15449485), (0, 0.0), (3, 0.0), (4, 0.0)]


# Zad 

Teraz proszę zrobić to samo z LSI (użyj **num_topics=4**).

In [35]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=4)
vec_lsi = lsi[vec_bow] # convert the query to LSI space

index = similarities.MatrixSimilarity(lsi[corpus])

sims = index[vec_lsi] # perform a similarity query against the corpus
print(list(enumerate(sims)))


[(0, 0.7177205), (1, 0.596951), (2, 0.9408147), (3, 0.0), (4, 0.0)]


In [36]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims)

[(2, 0.9408147), (0, 0.7177205), (1, 0.596951), (3, 0.0), (4, 0.0)]


#### Proszę zauważyć że w LSI pasujące dokumenty nie muszą zawierać słów które występuja w query. Dlaczego?